In [1]:
import geopandas as gpd

## Open Denny's Data

In [9]:
dennys_gdf = gpd.read_file("data/dennys_per_state.gpkg")
dennys_gdf = dennys_gdf[["STUSPS", "DENNYS"]]

In [10]:
gpd.read_file("data/dennys_per_state.gpkg")[["NAME", "per_1m"]].sort_values(
    "per_1m", ascending=False
).head(3)

,NAME,per_1m
12,New Mexico,13.671739
28,Nevada,12.444145
35,Arizona,10.571352


In [11]:
gpd.read_file("data/dennys_per_state.gpkg")[["NAME", "DENNYS"]].sort_values(
    "DENNYS", ascending=False
).head(2)

,NAME,DENNYS
16,California,355.0
15,Texas,195.0


## Open IHOP Data

In [12]:
ihops_gdf = gpd.read_file("data/IHOPs_per_state.gpkg")
ihops_gdf = ihops_gdf[["geometry", "STUSPS", "IHOPs"]]

In [13]:
gpd.read_file("data/IHOPs_per_state.gpkg")[["NAME", "per_1m"]].sort_values(
    "per_1m", ascending=False
).head(3)

,NAME,per_1m
26,Kansas,10.164323
38,Maryland,8.685775
2,Oklahoma,8.612455


In [14]:
gpd.read_file("data/IHOPs_per_state.gpkg")[["NAME", "IHOPs"]].sort_values(
    "IHOPs", ascending=False
).head(2)

,NAME,IHOPs
16,California,223.0
15,Texas,216.0


## Open Wafflehouse data

In [23]:
wh_gdf = gpd.read_file("data/whs_per_state.gpkg")
wh_gdf = wh_gdf[["STUSPS", "WHS"]]

In [29]:
gpd.read_file("data/whs_per_state.gpkg")[["NAME", "per_1m"]].sort_values(
    "per_1m", ascending=False
).head(3)

,NAME,per_1m
18,Georgia,40.218965
33,South Carolina,32.666213
17,Alabama,31.068808


In [25]:
gpd.read_file("data/whs_per_state.gpkg")[["NAME", "WHS"]].sort_values(
    "WHS", ascending=False
).head(2)

,NAME,WHS
18,Georgia,445.0
9,Florida,190.0


#### Merge Data now

In [36]:
dennys_ihop_gdf = ihops_gdf.merge(dennys_gdf, on="STUSPS", how="left")
dennys_ihop_wh_gdf = dennys_ihop_gdf.merge(wh_gdf, on="STUSPS", how="left")

In [37]:
dennys_ihop_wh_gdf["IHOPs"] = dennys_ihop_wh_gdf["IHOPs"].astype(int)
dennys_ihop_wh_gdf["DENNYS"] = dennys_ihop_wh_gdf["DENNYS"].astype(int)
dennys_ihop_wh_gdf["WHS"] = dennys_ihop_wh_gdf["WHS"].astype(int)

In [ ]:
## Doesn't work if

In [38]:
dennys_ihop_wh_gdf["MOST_STORES"] = dennys_ihop_wh_gdf[
    dennys_ihop_wh_gdf.columns[2:]
].idxmax(axis=1)

In [40]:
def more_stores(row) -> str:
    dennys = row["DENNYS"]
    ihop = row["IHOPs"]
    wh = row["WHS"]
    if dennys > ihop and dennys > wh:
        return "More Denny's"
    if ihop > dennys and ihop > wh:
        return "More IHOPs"
    if wh > dennys and wh > ihop:
        return "More WHs"
    else:
        return "Edge Case"

In [42]:
dennys_ihop_wh_gdf["label"] = dennys_ihop_wh_gdf.apply(
    lambda row: more_stores(row), axis=1
)

In [44]:
dennys_ihop_wh_gdf[dennys_ihop_wh_gdf["label"] == "Edge Case"]

,geometry,STUSPS,IHOPs,DENNYS,WHS,MOST_STORES,label
8,"MULTIPOLYGON (((-1357016.205 78008.02, -135633...",ID,7,7,0,IHOPs,Edge Case
36,"MULTIPOLYGON (((1950799.304 -402452.215, 19512...",DC,2,2,0,IHOPs,Edge Case
47,"MULTIPOLYGON (((-296121.109 437082.483, -29610...",ND,3,3,0,IHOPs,Edge Case


## Fix Edge cases

In [46]:
def more_stores(row) -> str:
    dennys = row["DENNYS"]
    ihop = row["IHOPs"]
    wh = row["WHS"]
    if dennys > ihop and dennys > wh:
        return "More Denny's"
    if ihop > dennys and ihop > wh:
        return "More IHOPs"
    if wh > dennys and wh > ihop:
        return "More WHs"
    if ihop == dennys:
        return "Same Amount"
    else:
        return "Edge Case"

In [47]:
dennys_ihop_wh_gdf["label"] = dennys_ihop_wh_gdf.apply(
    lambda row: more_stores(row), axis=1
)

In [48]:
dennys_ihop_wh_gdf[dennys_ihop_wh_gdf["label"] == "Edge Case"]

,geometry,STUSPS,IHOPs,DENNYS,WHS,MOST_STORES,label


In [49]:
dennys_ihop_wh_gdf = dennys_ihop_wh_gdf.to_crs(9311)
dennys_ihop_wh_gdf.to_file("data/ihop_dennys_wh.gpkg")